In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import geopandas as gpd
import pandas as pd
import requests
import zipfile
import io
import tempfile
import os
from shapely import wkt
import csv

# 0. Make database of tile indices (only need to do this once)

## Get all shapefile URLs from metadata page

In [2]:
#URL for the tile index folder
base_url = "https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/index.html"

# Fetch the page content
response = requests.get(base_url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch URL: {base_url}")

#Parse content
soup = BeautifulSoup(response.text, 'html.parser')

# Collect all ZIP file URLs
tile_index_urls = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href and href.endswith('.zip'):
        full_url = urljoin(base_url, href)
        tile_index_urls.append(full_url)

# Output the list of tile index ZIP URLs (check one or two)
print("Tile index shapefile URLs:")
for url in tile_index_urls:
    print(url)

Tile index shapefile URLs:
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10196/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10175/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10236/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10273/tileindex_2023_ngs_topobathy_longIslandSound_m10273.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10112/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid12b/10045/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9857/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9689/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9570/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9687/tileindex.zip
https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9651/tileindex.zip
https://noa

## Make GeoDataFrame with shapefile URLs, file names and indices (takes a while)

In [7]:
# List to hold individual GeoDataFrames
gdf_list = []

for url in tile_index_urls:
    print(f"Processing: {url}")
    # Download the zip file
    response = requests.get(url)
    if response.status_code == 200:
        # Create a temporary directory to extract the zip
        with tempfile.TemporaryDirectory() as tmpdirname:
            # Read the zipfile from the response content
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall(tmpdirname)
                
                # Identify the shapefile (.shp) within the extracted files.
                shp_files = [os.path.join(tmpdirname, f) for f in os.listdir(tmpdirname) if f.endswith('.shp')]
                if shp_files:
                    shp_path = shp_files[0]
                    # Read the shapefile into a GeoDataFrame
                    gdf = gpd.read_file(shp_path)
                    gdf = gdf.to_crs("EPSG:4326")
                    
                    # Inspect available columns (uncomment the next line to print the column names)
                    print(gdf.columns.tolist())
                    
                    # Adjust the field names if necessary.
                    if 'filename' in gdf.columns:
                        gdf = gdf.rename(columns={
                        'filename': 'name', 
                        'URL': 'url',
                        'Index':'index', 
                    })
                    else:
                        gdf = gdf.rename(columns={
                            'Name': 'name', 
                            'URL': 'url',
                            'Index':'index', 
                        })
                    
                    # Select only the columns of interest along with the geometry.
                    fields = ['geometry', 'name', 'url', 'index']
                    # Check that all required fields exist in the GeoDataFrame.
                    missing = [field for field in fields if field not in gdf.columns]
                    if missing:
                        print(f"Warning: Missing expected fields {missing} in {url}")
                    else:
                        gdf = gdf[fields]
                    
                    gdf_list.append(gdf)
                else:
                    print(f"No shapefile found in the zip from {url}")
    else:
        print(f"Failed to download: {url}")

# Combine all the GeoDataFrames into one
if gdf_list:
    combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
    print("Combined GeoDataFrame:")
    print(combined_gdf.head())
else:
    print("No valid GeoDataFrames were loaded.")


Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10196/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10175/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10236/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10273/tileindex_2023_ngs_topobathy_longIslandSound_m10273.zip
['filename', 'srs', 'url', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/10112/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid12b/10045/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/9857/tileindex.zip
['Index', 'Name', 'URL', 'geometry']
Processing: https://noaa-

KeyboardInterrupt: 

## Save as CSV

In [6]:
# Clean up the combined GeoDataFrame
combined_gdf_clean = combined_gdf[['url', 'name', 'geometry']].copy()

# Remove any embedded newlines from the url field
combined_gdf_clean['url'] = combined_gdf_clean['url'].astype(str).str.replace(r'[\r\n]+', '', regex=True)

# Convert geometry to WKT (string form)
combined_gdf_clean['geometry'] = gpd.GeoSeries(combined_gdf_clean['geometry']).to_wkt()

# Remove any embedded newlines from the geometry string, if any
combined_gdf_clean['geometry'] = combined_gdf_clean['geometry'].str.replace(r'[\r\n]+', ' ', regex=True)

# define output path
output_csv = 'data_table.csv'

# Save as CSV
combined_gdf_clean.to_csv(output_csv, index=False)

/var/folders/ch/s01gcd2563d22fc8stwr0dfm0000gn/T/ipykernel_48217/1978997146.py:8: UserWarning: Geometry column does not contain geometry.
  combined_gdf_clean['geometry'] = gpd.GeoSeries(combined_gdf_clean['geometry']).to_wkt()
/var/folders/ch/s01gcd2563d22fc8stwr0dfm0000gn/T/ipykernel_48217/1978997146.py:11: UserWarning: Geometry column does not contain geometry.
  combined_gdf_clean['geometry'] = combined_gdf_clean['geometry'].str.replace(r'[\r\n]+', ' ', regex=True)


# 1. Query data table to get files of interest into smaller CSV (repeat 1. and 2. as needed for different regions as needed)

In [9]:
# Working folder
folder = ''

# Load CSV
df = pd.read_csv(folder + 'data_table.csv')

# Convert WKT to geometry
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
gdf = gdf.to_crs("EPSG:3857")

## Use spatial joins to query data

### Example: Querying all datasets within a 2 km buffer of the NOAA CUSP shorelines

In [ ]:
# Get CUSP shoreline data at https://nsde.ngs.noaa.gov (download shapefiles for appropriate area and modify path as needed)
coast = "North_Atlantic"
coast_path = folder + "CUSP shorelines/" + coast + "/"
shoreline_path = coast_path + coast + ".shp"
shoreline_gdf = gpd.read_file(shoreline_path)
shoreline_gdf.crs  = "EPSG:4269"
shoreline_gdf_m = shoreline_gdf.to_crs("EPSG:3857") # convert to CRS that works in meters

# Create buffer around coastlines
coastline_buffer = shoreline_gdf_m.buffer(2000) # 2km buffer
coastline_buffer.to_file(coast_path + coast +"_2km_buffer.shp") # save buffers as shapefile
dissolved_buffer = coastline_buffer.union_all() # dissolve the buffer polygons  
dissolved_gdf = gpd.GeoDataFrame(geometry=[dissolved_buffer], crs=coastline_buffer.crs)
dissolved_gdf = dissolved_gdf.to_crs("EPSG:4326")
dissolved_gdf.to_file(coast_path + coast + "_2km_buffer_dissolved.shp") # save dissolved buffers as shapefile

In [ ]:
# Define latitude and longitude boundaries
min_lat, max_lat = 29, 31
min_lon, max_lon = -91.5, -87.5

# Filter tiles by centroid lat/lon first
tiles_bbox_filtered = gdf[
    (gdf.geometry.centroid.y >= min_lat) &
    (gdf.geometry.centroid.y <= max_lat) &
    (gdf.geometry.centroid.x >= min_lon) &
    (gdf.geometry.centroid.x <= max_lon)
]

print(f"{len(tiles_bbox_filtered)} tiles after lat/lon bounding box filtering.")

# Next, apply spatial intersection with coastline buffer
tiles_final_filtered = tiles_bbox_filtered[
    tiles_bbox_filtered.intersects(dissolved_gdf.geometry[0])
]

print(f"{len(tiles_final_filtered)} tiles intersect the coastal buffer after bbox filtering.")

In [ ]:
query_area_name = "nola"
tiles_final_filtered.to_csv(query_area_name + '_data.csv')

# 2. Use terminal and bash script to download elements in query table from s3 bucket

    ./download_script.sh nola_data.csv